# Tracking Trump: Electoral Votes

[Morning Consult](https://morningconsult.com) has a page called [Tracking Trump](https://morningconsult.com/tracking-trump/) that summarizes the presidential approval polls on a state-by-state basis, and tells you how many states Trump has a net positive or net negative approval rating. But I don't care about the number of *states*; I care about the number of *electoral votes*. This notebook computes that for me:

In [1]:
import urllib.request
import re
from collections import namedtuple, defaultdict

Row = namedtuple('Row', 'state delta jan17app jan17dis jan17err app dis err')

def read_data(url='https://morningconsult.com/tracking-trump/'):
    "Fetch data from the website and return a list of `Row` objects."
    with urllib.request.urlopen(url) as response:
        html = response.read().decode('utf-8')
        rows = re.findall(r'<tr(.*?)</tr>', html, re.S)
        return [parse_row(row) for row in rows[1:]]
    
def parse_row(row):
    "Parse an html string into a `Row` object."
    state, *nums =  re.findall('>([^>]*?)</td', row.replace('%', ''))
    return Row(state, *map(int, nums))

def net(row): "Approval minus disapproval."; return row.app - row.dis 

rows = read_data()

def EV(swing=0, rows=rows):
    "How many electoral votes would Trump win today, given a swing."
    return sum(votes[row.state] for row in rows if net(row) + swing > 0)

votes = dict(AL=9,  AK=3,  AZ=11, AR=6,  CA=55, CO=9,  CT=7, DE=3,  DC=3,  FL=29, GA=16, HI=4, ID=4,
             IL=20, IN=11, IA=6,  KS=6,  KY=8,  LA=8,  ME=4, MD=10, MA=11, MI=16, MN=10, MS=6, MO=10, 
             MT=3,  NE=5,  NV=6,  NH=4,  NJ=14, NM=5, NY=29, NC=15, ND=3,  OH=18, OK=7,  OR=7, PA=20, 
             RI=4,  SC=9,  SD=3,  TN=11, TX=38, UT=6, VT=3,  VA=13, WA=12, WV=5,  WI=10, WY=3)

In [2]:
EV()

164

What that says is that if an election were held today, January 23, 2019, and if Trump won those states in which he has a net positive approval (i.e. more approval than disapproval), **he would get 164 electoral votes** (that's 9 worse than McCain in 2008). You need **270** to win.

But things can change. The election is a long ways away, and we don't know who's running, and we don't know if there is systematic bias in the polling data. In the table below, I list the number of electoral votes Trump would get assuming he gets an increase of net approval of 0 to 9 percentage points across the board in every state. We see he would need a **7** percent upswing in order to win.

In [3]:
{m: EV(m) for m in range(10)}

{0: 164,
 1: 164,
 2: 164,
 3: 164,
 4: 255,
 5: 255,
 6: 255,
 7: 279,
 8: 279,
 9: 294}

Here are the states sorted by current net approval:

In [4]:
sorted((net(row), row.state, votes[row.state]) 
       for row in rows)

[(-62, 'DC', 3),
 (-31, 'VT', 3),
 (-29, 'CA', 55),
 (-29, 'MA', 11),
 (-28, 'HI', 4),
 (-28, 'MD', 10),
 (-23, 'NY', 29),
 (-23, 'WA', 12),
 (-22, 'OR', 7),
 (-21, 'IL', 20),
 (-20, 'CT', 7),
 (-19, 'NM', 5),
 (-19, 'RI', 4),
 (-18, 'NJ', 14),
 (-16, 'CO', 9),
 (-16, 'DE', 3),
 (-14, 'MN', 10),
 (-12, 'MI', 16),
 (-12, 'WI', 10),
 (-11, 'NV', 6),
 (-10, 'IA', 6),
 (-8, 'AZ', 11),
 (-8, 'NH', 4),
 (-6, 'ME', 4),
 (-6, 'PA', 20),
 (-3, 'FL', 29),
 (-3, 'GA', 16),
 (-3, 'NC', 15),
 (-3, 'OH', 18),
 (-3, 'VA', 13),
 (2, 'KS', 6),
 (3, 'MO', 10),
 (3, 'UT', 6),
 (4, 'AK', 3),
 (4, 'MT', 3),
 (4, 'TX', 38),
 (8, 'IN', 11),
 (8, 'NE', 5),
 (9, 'ND', 3),
 (9, 'SC', 9),
 (11, 'AR', 6),
 (11, 'ID', 4),
 (13, 'LA', 8),
 (13, 'OK', 7),
 (15, 'KY', 8),
 (15, 'SD', 3),
 (16, 'TN', 11),
 (17, 'MS', 6),
 (21, 'AL', 9),
 (26, 'WV', 5),
 (32, 'WY', 3)]